![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [31]:
# Start your code here!
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv('customer_train.csv')
print(ds_jobs.head())

# Copy of the dataframe for cleaning
ds_jobs_clean = ds_jobs.copy()
# Checking the dtypes of the columns
print(ds_jobs_clean.dtypes)

# Converting the original dtypes to the appropriate dtype

# Converting int64 to int32
ds_jobs_clean[['student_id','training_hours','job_change']] = ds_jobs_clean[['student_id','training_hours','job_change']].astype('int32')
print(ds_jobs_clean[['student_id','training_hours','job_change']].dtypes)

# Converting float64 to float16 to reduce memory usage
ds_jobs_clean['city_development_index'] = ds_jobs_clean['city_development_index'].astype('float16')
print(ds_jobs_clean['city_development_index'].dtypes)

# Converting nominal categorical data to category dtype
ds_jobs_clean[['gender','city','major_discipline','company_type']] = ds_jobs_clean[['gender','city','major_discipline','company_type']].astype('category')
print(ds_jobs_clean['gender'].dtypes)

# Converting ordinal categorical data to categorical dtype
x = ds_jobs_clean['education_level'].value_counts()
y = ds_jobs_clean['company_size'].value_counts()
z = ds_jobs_clean['company_type'].value_counts()
a = ds_jobs_clean['last_new_job'].value_counts()

print(x, y, z, a)

# Ordered categories
ds_jobs_clean['relevant_experience'] = pd.Categorical(ds_jobs_clean['relevant_experience'], categories=['No relevant experience','Has relevant experience'], ordered=True)
ds_jobs_clean['enrolled_university'] = pd.Categorical(ds_jobs_clean['enrolled_university'], categories=['no_enrollment','Part time course', 'Full time course'], ordered=True)
ds_jobs_clean['education_level'] = pd.Categorical(ds_jobs_clean['education_level'], categories=['Primary School','High School','Graduate','Masters','Phd'], ordered=True)
ds_jobs_clean['experience'] = pd.Categorical(ds_jobs_clean['experience'], categories=['<1']+list(map(str,range(1,21)))+['>20'], ordered=True)
ds_jobs_clean['company_size'] = pd.Categorical(ds_jobs_clean['company_size'], categories=['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'], ordered=True)
ds_jobs_clean['last_new_job'] = pd.Categorical(ds_jobs_clean['last_new_job'], categories=['never','1','2','3','4','>4'], ordered=True)

# Convert to category dtype
ds_jobs_clean[['relevant_experience','enrolled_university','education_level','experience','company_size','last_new_job']] = ds_jobs_clean[['relevant_experience','enrolled_university','education_level','experience','company_size','last_new_job']].astype('category')
print(ds_jobs_clean.dtypes)

# Dataframe containing only students with 10 or more years of experience and companies with at least 1000 employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]
print(ds_jobs_clean.head())

print(ds_jobs_clean.head())
print(ds_jobs.memory_usage())
print(ds_jobs_clean.memory_usage())

   student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36          1
1       29725   city_40  ...              47          0
2       11561   city_21  ...              83          0
3       33241  city_115  ...              52          1
4         666  city_162  ...               8          0

[5 rows x 14 columns]
student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object
student_id        int32
training_hours    int32
job_change        int32
dtype: object
float16
category
Graduate          11598
Masters            4361
